In [14]:
#Libary..  
from ortools.sat.python import cp_model
from pydantic import BaseModel
from typing import List

In [15]:
# Define input data models
class Task(BaseModel):
    id: int
    name: str
    priority: int
    duration: int
    deadline: int
    
class Schedule(BaseModel):
    task_id: int
    start_time: int
    end_time: int
    
class SchedulerInput(BaseModel):
    tasks: List[Task]
    max_time: int

In [79]:
def optimize_schedule(input_data):
    model = cp_model.CpModel()
    max_time = input_data.max_time
    tasks = input_data.tasks

    # Decision variables
    start_vars = {}
    end_vars = {}

    for task in tasks:
        start_vars[task.id] = model.NewIntVar(0, max_time, f'start_{task.id}')
        end_vars[task.id] = model.NewIntVar(0, max_time, f'end_{task.id}')
        model.Add(end_vars[task.id] == start_vars[task.id] + task.duration)
        model.Add(end_vars[task.id] <= task.deadline)  # Deadline constraint

    # Create interval variables (fixing the missing argument)
    intervals = []
    for task in tasks:
        interval_var = model.NewIntervalVar(
            start_vars[task.id],   # Start time variable
            task.duration,         # Task duration
            end_vars[task.id],     # End time variable
            f'interval_{task.id}'  # Name of the interval variable
        )
        intervals.append(interval_var)

    # No overlapping tasks
    model.AddNoOverlap(intervals)

    # Solve the model and get the actual start and end times
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # Extract and return the scheduled tasks with start and end times
        scheduled_tasks = []
        for task in tasks:
            scheduled_task = {
                "task_id": task.id,
                "start_time": solver.Value(start_vars[task.id]),
                "end_time": solver.Value(end_vars[task.id])
            }
            scheduled_tasks.append(scheduled_task)
        return scheduled_tasks
    else:
        raise Exception("No feasible solution found.")


In [82]:
def test_single_task_within_deadline():
    # Setup: One task with duration 2 and deadline 3
    tasks = [Task(id=1, name="Task 1", priority=1, duration=2, deadline=3)]
    input_data = SchedulerInput(tasks=tasks, max_time=1)
    # Call the function
    schedule = optimize_schedule(input_data)
    
    # Ensure at least one task is scheduled
    assert len(schedule) == 1
    task = schedule[0]
    # Verify the task's timing constraints
    assert task["task_id"] == 1
    assert task["end_time"] == task["start_time"] + 2  # Duration check
    assert task["end_time"] <= 3  # Deadline constraint check
    print("Scheduled Task:", task)

In [83]:
test_single_task_within_deadline()

Exception: No feasible solution found.